# Laboratorio 3

## Tiara Ramírez - Valentina Cáceres

## 1. Arquitectura de la CNN 

<p style="text-align: justify; line-height: 1.5;">
En el codigo se utiliza la arquitectura de GoogLeNet (o Inception V1) que es un modelo de Deep Learning diseñado para tareas de clasificacion de imagene, que cuenta con 22 capas en total. Se cuenta con un modelo de Machine Learning mas profundo que otros anteriores, lo que implica tener más capacidad de apredizaje y, como resultado, esto aumenta el rendimiento de del modelo.  
</p>

### Modelo de Deep Learning (aprendizaje profundo): 
<p style="text-align: justify; line-height: 1.5;">
GoogLeNet tiene 22 capas profundas, pero se encuentra optimizado para reducir su complejedad utilizando bloques Inception, los cuales cuentan con una combinacion de diferentes tamaños de filtros de convolución en paralelo, como se ve en la imagen:
</p>

<div style="text-align: center;">
    <img src="Imagenes_lab3\google-net-like.png" width="40%">
</div>

<p style="text-align: justify; line-height: 1.5;">
Donde se tiene:  
<br>
<b>Input:</b> Corresponde una imagen de tamaño 30x30, se sabe que cuenta con 3 canales de color debido a la profundidad.   <br>
<b>C1:</b> Corresponde a la primera capa de convolución que aplica un filtro 5x5, reduciendo el tamaño de la imagen a 28x28 con una profundidad de 10. Aquí se extraen las características básicas de cada imagen entrante, como lo son los bordes y las texturas.   <br>
<b>P1:</b> Capa de pooling (combinación), en esta capa se reduce el tamaño utilizando un filtro de 2x2, disminuyendo a 14x14. Esta capa ayuda a reducir el número de parámetros, pero manteniendo las características más importantes. <br>
<b>C2, C3, y C4:</b> Son capas de convolución adicionales que aplican filtros para continuar extrayendo las características más complejas en cada paso. Se puede ver cómo las dimensiones van disminuyendo de forma progresiva en cada capa, a diferencia de la profundidad, que va aumentando.   <br>
<b>P2:</b> Segunda capa de pooling, que se encarga de disminuir el tamaño a 7x7 con una profundidad de 18 antes de entrar al bloque Inception.   <br>
<b>Bloque Inception:</b>  Dentro de este bloque se aplican múltiples filtros de diferentes tamaños (1x1, 3x3, 5x5 y max pooling) en paralelo a la entrada. Así la red puede capturar características a diferentes escalas y niveles de detalle. Esto se puede ver de forma gráfica en la siguiente imagen.
</p> 

<div style="text-align: center;">
    <img src="Imagenes_lab3\Incepption-module.png" width="40%">
</div>

<p style="text-align: justify; line-height: 1.5;">
<b>P3: </b> Tercera capa de pooling, se encarga de reducir la dimensión espacial a 3x3, manteniendo la profundidad en 80.  <br>
<b>M1: </b> Capa de acoplamiento (o flatten), que convierte la salida de un vector unidimensional para la capa completamente conectada.  <br>
<b>FC (Fully Connected): </b> Capa completamente conectada que contiene 80 neuronas. Cada neura está conectada con todas las características obtenidas en los puntos anteriores.<br>
<b>Output: </b> La salida de la red es una capa de clasificación que determina la probabilidad de cada clase  <br>
</p> 

#### Modificación del modelo
<p style="text-align: justify; line-height: 1.5;">
Es necesario adaptar el modelo al problema planteado en el laboratorio, es decir a las imagenes que se deben clasifican en los 21 tipos de uso de suelo (21 clases) obtenidas de la base de datos a utilizar. La adaptación se realiza de forma simple cambiando en el codigo inferior para que la red pueda clasificar en 21 clases en lugar de las 1000 clases originales de ImageNet.<br>
</p>

### Transfer Learning 
<p style="text-align: justify; line-height: 1.5;">
El transfer learning es una técnica utilizada en el entorno de la inteligencia artificial, donde se aprovecha los pesos preentrenados de un modelo en un conjunto de datos grandes, como ImageNet, y se ajustan para resolver un problema específico con un conjunto de datos mas pequeño o diferente.<br>
Esto tiene diversas ventajas, como lo son: aumento en la eficiencia del entrenamiento, menor necesidad de datos y unos mejores resultados en el entrenamiento. <br>
En este caso, se utiliza un transfer learning parcial, donde se reutilizan los pesos preentrenados en las capas convolucionales de GoogLeNet, y unicamente se ajustan las capas finales del modelo para adaptarlo al problema de las 21 clases explicado con anterioridad.
</p>

### Entrenamiento del modelo
<p style="text-align: justify; line-height: 1.5;">
En este caso el modelo se entrena durante 10 épocas y en cada iteracion se guardan los parámetros ajustados. Generando 10 archivos .pth, uno por cada época, que contienen los pesos actualizados. Lo que permite realizar validaciones a modelos intermedios para determinar por ejemplo que modelo presentó el mejor desempeño, y para evitar perder el progreso si se llega a interrumpir el entrenamiento, lo que es muy util devido a lo tardado del proceso.<br>
Se aclara ademas que en este caso la función de perdida utilizada corresponde a CrossEntropyLoss, que es la adecuada para los problemas de clasificacion con varias clases.
</p>

## 2. Entrenamiento del modelo con la base de datos de "Uso de suelo":

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np

# Set device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")               # Selección de la GPU
print("cuda" if torch.cuda.is_available() else "cpu")                               # Imprime el dispositivo seleccionado

# Load pretrained GoogLeNet model
model = models.googlenet(pretrained=True)                                           # Carga el modelo preentrenado GoogLeNet

# Modify the final fully connected layer for 100 classes
num_classes = 21                                                                    # Número de clases                      
model.fc = nn.Linear(model.fc.in_features, num_classes)                             # Modifica la capa final del modelo

# Move model to the chosen device
model = model.to(device)                                                            # Mueve el modelo a la GPU

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()                                                   # Define la función de pérdida
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)                          # Define el optimizador

# Data transformations (customize as needed)

    #Sin Data Augmentation
# transform= transforms.Compose([                                                    # Transformaciones de los datos
# transforms.Resize((224, 224)),                                                      # Redimensiona las imágenes
# transforms.ToTensor(),                                                              # Convierte las imágenes a tensores
# transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),        # Normaliza los tensores
# ])

    #Con Data Augmentation 
transform = transforms.Compose([                                                    # Transformaciones de los datos
transforms.Resize((224, 224)),                                                      # Redimensiona las imágenes
transforms.RandomHorizontalFlip(p=0.5),                                             # Voltea horizontalmente las imágenes con una probabilidad del 50%
transforms.RandomRotation(10),                                                      # Rota las imágenes aleatoriamente en un rango de 10 grados
transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),      # Ajusta el brillo, contraste, saturación y tono de las imágenes
transforms.ToTensor(),                                                              # Convierte las imágenes a tensores
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),        # Normaliza los tensores
])

# Load data
train_dataset = datasets.ImageFolder(root="C:\\Users\\tiari\\OneDrive\\Documents\\IA\\images_train_test_val\\train", transform=transform)       # Carga el conjunto de datos de entrenamiento
val_dataset = datasets.ImageFolder(root="C:\\Users\\tiari\\OneDrive\\Documents\\IA\\images_train_test_val\\validation", transform=transform)    # Carga el conjunto de datos de validación

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)               # Carga el conjunto de datos de entrenamiento
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)                  # Carga el conjunto de datos de validación


# Set up early stopping 
patience = 5
best_loss = np.inf
counter = 0

# Training loop
num_epochs = 50                                                                     # Número de épocas
# Bucle de entrenamiento
for epoch in range(num_epochs):
    model.train()                                                                   # Pone el modelo en modo de entrenamiento
    running_loss = 0.0                                                              # Inicializa 
    for images, labels in train_loader:                                             # Bucle de entrenamiento
        images, labels = images.to(device), labels.to(device)                       # Mueve las imágenes y las etiquetas a la GPU

        # Forward pass
        outputs = model(images)                                                     # Pasa las imágenes por el modelo
        loss = criterion(outputs, labels)                                           # Calcula la pérdida

        # Backward pass and optimization
        optimizer.zero_grad()                                                       # Reinicia los gradientes
        loss.backward()                                                             # Retropropagación
        optimizer.step()                                                            # Actualiza los pesos
        running_loss += loss.item()                                                 # Acumula la pérdida
        
    model.eval()                                                                    # Pone el modelo en modo de evaluación
    val_loss = 0.0                                                                  # Inicializa
    with torch.no_grad():                                                           # Desactiva el cálculo de gradientes
        for images, labels in val_loader:                                           # Bucle de validación
            images, labels = images.to(device), labels.to(device)                   # Mueve las imágenes y las etiquetas a la GPU

            # Forward pass
            outputs = model(images)                                                 # Pasa las imágenes por el modelo
            loss = criterion(outputs, labels)                                       # Calcula la pérdida
            val_loss += loss.item()                                                 # Acumula la pérdida
    val_loss /= len(val_loader)                                                     # Calcula la pérdida promedio
        
        
    print(f"Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(train_loader):.4f}, Validation Loss: {val_loss:.4f}", end="")
    
    #Early stopping
    if val_loss < best_loss:
        best_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), f"C:\\Users\\tiari\\OneDrive\\Documents\\IA\\modelos\\\\model_con_da_epoch_{epoch+1}.pth")
        print("(Guardado)")
    else:
        counter += 1
        print(f"Stop: {counter}/{patience}")
        if counter >= patience:
            print("Early stop.")
            break

print("Training complete.")

### a. Sin Data Augmentation.
<p style="text-align: justify; line-height: 1.5;">
Se comienza entrenando el modelo sin utilizar técnicas de data augmentation, por lo tanto los conjuntos de entrenamiento solo realizan las transformaciones basicas necesarias para ajustar las imagenes del conjunto de entrenamiento a lo requerido por GoogLeNet y normalizar los valores de los pixeles basados en las estadisticas del conjunto de datos ImageNet. Esto se puede observar en el codigo dedicado al entrenamiento (superior).<br>
Obteniendo la siguiente salida al terminar el entrenamiento:
</p>


<p style="text-align: center; line-height: 1.5;">
Epoch [1/50], Training Loss: 0.6966, Validation Loss: 0.5315 (Guardado)<br> 
Epoch [2/50], Training Loss: 0.2750, Validation Loss: 0.4394 (Guardado)<br>
Epoch [3/50], Training Loss: 0.2123, Validation Loss: 0.2174 (Guardado)<br>
Epoch [4/50], Training Loss: 0.1602, Validation Loss: 0.3055 (Stop 1/5)<br>
Epoch [5/50], Training Loss: 0.1286, Validation Loss: 0.3191 (Stop 2/5)<br>
Epoch [6/50], Training Loss: 0.1587, Validation Loss: 0.3480 (Stop 3/5)<br>
Epoch [7/50], Training Loss: 0.1014, Validation Loss: 0.1829 (Guardado)<br>
Epoch [8/50], Training Loss: 0.0998, Validation Loss: 0.0860 (Guardado)<br>
Epoch [9/50], Training Loss: 0.0897, Validation Loss: 0.2150 (Stop 1/5)<br>
Epoch [10/50], Training Loss: 0.0777, Validation Loss: 0.1037 (Stop 2/5)<br>
Epoch [11/50], Training Loss: 0.0738, Validation Loss: 0.1684 (Stop 3/5)<br>
Epoch [12/50], Training Loss: 0.0540, Validation Loss: 0.1133 (Stop 4/5)<br>
Epoch [13/50], Training Loss: 0.0807, Validation Loss: 0.1581 (Stop 5/5)<br>
</p>

<p style="text-align: justify; line-height: 1.5;">
El entrenamiento se detuvo después de la época 13 debido a la condición de paciencia (early stopping) al alcanzar 5 iteraciones consecutivas sin mejora en la validación.<br>
Además se puede ver a partir de la disminucion de la pérdida en cada época que el modelo es capáz de aprender representaciones utiles de cada clase de la base de datos, sin la necesidad de utilizar data augmentation. 

</p>

### b. Con Data Augmentation
<p style="text-align: justify; line-height: 1.5;">
Para agregar tecnicas de Data Augmentation y así aumentar la robustez del modelo, se realizan diversas transformaciones (Invertir, rotar, variar brillo, color y tono), generando variaciones de las imágenes originales. Esto ayude a evitar sobreajustes y mejora la capacidad de generalización del modelo. <br>
Obteniendo en la salida:
</p>

<p style="text-align: center; line-height: 1.5;">
Epoch [1/50], Training Loss: 0.7105, Validation Loss: 0.4326 (Guardado) <br>
Epoch [2/50], Training Loss: 0.3750, Validation Loss: 0.3583 (Guardado)<br>
Epoch [3/50], Training Loss: 0.2506, Validation Loss: 0.2232 (Guardado)<br>
Epoch [4/50], Training Loss: 0.2197, Validation Loss: 0.2563 (Stop 1/5)<br>
Epoch [5/50], Training Loss: 0.2074, Validation Loss: 0.1248 (Guardado)<br>
Epoch [6/50], Training Loss: 0.1735, Validation Loss: 0.2542 (Stop 1/5)<br>
Epoch [7/50], Training Loss: 0.1349, Validation Loss: 0.3105 (Stop 2/5)<br>
Epoch [8/50], Training Loss: 0.1353, Validation Loss: 0.0885 (Guardado)<br>
Epoch [9/50], Training Loss: 0.1225, Validation Loss: 0.1572 (Stop 1/5)<br>
Epoch [10/50], Training Loss: 0.0991, Validation Loss: 0.1222 (Stop 2/5)<br>
Epoch [11/50], Training Loss: 0.1040, Validation Loss: 0.1244 (Stop 3/5)<br>
Epoch [12/50], Training Loss: 0.1321, Validation Loss: 0.1980 (Stop 4/5)<br>
Epoch [13/50], Training Loss: 0.1106, Validation Loss: 0.2816 (Stop 5/5)<br>
</p>


Se puede ver que al igual en el caso anterior, durante las primeras épocas la perdida disminuye rápidamente, lo que indica que el modelo logra aprender de los datos generados, deteniendose después de la época 13 debido al criterio de paciencia (early stopping).

### Comparacion de ambas técnicas.

<p style="text-align: justify; line-height: 1.5;">
En la tabla inferior se puede observar una comparativa de las metricas principales obtenidas del entrenamiento con y sin data augmentation, algunas observaciónes son:<br>
<b>1- Perdida inicial:</b> Se puede ver que la perdida incial es mayor al utilizar data augmentation, que es lo esperado ya que esta técnica tiene una mayor complejidad principalmente debido a las diferentes transformaciones aplicadas a las imágenes previo al entrenamiento.<br>
<b>2- Mejor perdida de validación:</b> Aunque la perdida de validación no es tan bajo en el caso del modelo con data augmentation en comparación, el modelo demuestra mayor capacidad de generalización y consistencia frente a los datos de validación. Ya que si bien en el caso sin data augmentation el modelo convergió rápidamente, pero hubo problemas de sobreajuste en épocas posteriores (valores de Validation Loss más altos que el esperado a pesar de la caída en Training Loss).<br>
<b>3- Perdida final de entrenamiento:</b> En este caso el valor es nuevamente mayor con data augmentation que sin él, lo que se podría asumir como un resultado menos favorable pero esto no implica necesariamente que el modelo sea menos efectivo.<br>
<b>4- Perdida final de validación:</b> Con data augmentation, aunque la perdida de validación no es tan baja en comparación al sin data augmentation, el modelo demuestra mayor capacidad de generalización y consistencia frente a los datos de validación.<br>
<b>5- Variación: </b> En el caso donde se utiliza data augmentation se puede ver que las ultimas épocas presentan una pérdida mas oscilante (en ocaciones el valor de la época entrenada es superior al de la época anterior), esto se asume que es debido a que el modelo ya se está acercando al límite de su capacidad de aprendizaje para el conjunto de datos.<br>
<b>6- Generalización: </b> En este caso como el utilizar data augmentation fuerza al modelo a aprender caracteristicas mas robustas, ya que las diferentes transformaciones realizadas en el codigo anterior hacen que las imágenes sean más variadas y menos predecibles por asi decirlo. Esto en general mejora el desempeño en los conjuntos de validación, que es una prueba que se realizará mas adelante para comprobarlo.<br>
</p>


<div style="display: flex; justify-content: center;">
<table>
<tr>
        <th>Métrica</th>
        <th>Sin Data Augmentation</th>
        <th>Con Data Augmentation</th>
    </tr>
    <tr>
        <td>Pérdida Inicial</td>
        <td>0.6338</td>
        <td>0.7241</td>
    </tr>
    <tr>
        <td>Mejor pérdida de validación</td>
        <td>0.0860 (época 8)</td>
        <td>0.0885 (época 8)</td>
    </tr>
    <tr>
        <td>Pérdida Final de entrenamiento</td>
        <td>0.0540</td>
        <td>0.1106</td>
    </tr>
    <tr>
        <td>Pérdida Final de validación</td>
        <td>0.1581</td>
        <td>0.2816</td>
    </tr>
        <tr>
        <td>Número de épocas</td>
        <td>13</td>
        <td>13</td>
    </tr>
    <tr>
        <td>Variación de Pérdida</td>
        <td>Estable</td>
        <td>Más oscilante</td>
    </tr>
    <tr>
        <td>Generalización</td>
        <td>Menor (posible sobreajuste)</td>
        <td>Mejor (más robusto a datos nuevos)</td>
    </tr>
<table>
</div>

## 3. Evaluación de métricas.

### a. ¿Qué mecanismo de optimización utilizaron?
<p style="text-align: justify; line-height: 1.5;">
En esté caso se mantuvo el optimizador que ya se tenía en el codigo de referencia, que vendria siendo el optimizador <b>Adam</b> con un factor de aprendizade de 0.001. Adam es un algoritmo de optimizacion que combina las ventajas de dos diferentes optimizadores, RMSProp y SGD con momentum, ajustando dinamicamente el aprendizaje para cada parámetro en función de su gradiente.
</p>

### b. ¿Cuál es el criterio de detención del entrenamiento? 
<p style="text-align: justify; line-height: 1.5;">
Si bien el número maximo de épocas en que se puede entrenar el modelo es de 50, se tiene un mecanismo de detención temprana (early stopping) con una paciencia de 5 épocas. Esto implica que el proceso de entrenamiento se detendrá si la perdida de validación no mejora durante 5 épocas concecutivas, lo que permite optimizar el entrenamiento del modelo previendo sobreajustes innecesarios ni asignado un número insuficiente de epocas para la obtención de éste. El funcionamiento del early stoppin se pudo observar en el punto 2.
</p>

In [ ]:

import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt

# Configurar el dispositivo (usar GPU si está disponible)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")           # Selección de la GPU      
print("Dispositivo:", "cuda" if torch.cuda.is_available() else "cpu")           # Imprime el dispositivo seleccionado

# Cargar el modelo GoogLeNet preentrenado
model = models.googlenet(pretrained=True)                                       # Carga el modelo preentrenado GoogLeNet

# Modificar la capa totalmente conectada final para 21 clases
num_classes = 21                                                                # Número de clases
model.fc = nn.Linear(model.fc.in_features, num_classes)                         # Modifica la capa final del modelo

# Mover el modelo al dispositivo seleccionado
model = model.to(device)                                                        # Mueve el modelo a la GPU

# Cargar el mejor modelo guardado
model_path = "C:\\Users\\tiari\\OneDrive\\Documents\\IA\\modelos\\model_con_da_epoch_8.pth"     # Ruta del archivo .pth
model.load_state_dict(torch.load(model_path))                                   # Carga el estado del modelo
model.eval()                                                                    # Poner el modelo en modo de evaluación

# Definir las transformaciones de los datos (ajustar según sea necesario)
transform = transforms.Compose([                                                # Transformaciones de los datos
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Cargar el conjunto de pruebas
test_dataset = datasets.ImageFolder(root="C:\\Users\\tiari\\OneDrive\\Documents\\IA\\images_train_test_val\\test", transform=transform)    # Carga el conjunto de prueba
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)                                                                        # Carga el conjunto de prueba en un DataLoader

# Función para evaluar el modelo
def evaluate_model(model, test_loader):                                          # Función para evaluar el modelo
    all_labels = []                                                              # Inicializa las etiquetas verdaderas
    all_preds = []                                                               # Inicializa las etiquetas y las predicciones
    class_names = test_dataset.classes                                           # Obtiene los nombres de las clases

    with torch.no_grad():                                                        # Deshabilita el cálculo de gradientes
        # Bucle de evaluación
        for images, labels in test_loader:                                    
            images, labels = images.to(device), labels.to(device)               # Mueve las imágenes y las etiquetas a la GPU
            outputs = model(images)                                             # Pasa las imágenes por el modelo
            _, predicted = torch.max(outputs, 1)                                # Obtiene las predicciones
            all_labels.extend(labels.cpu().numpy())                             # Añade las etiquetas verdaderas
            all_preds.extend(predicted.cpu().numpy())                           # Añade las predicciones

    # Matriz de confusión
    cm = confusion_matrix(all_labels, all_preds)                                            # Calcula la matriz de confusión
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test_dataset.classes) # Configura la visualización
    disp.plot(cmap=plt.cm.Blues, xticks_rotation=90)                                        # Visualiza la matriz de confusión
    plt.title("Matriz de confusión")                                                        # Título de la visualización
    plt.show()                                                                              # Muestra la visualización
    
    # Indicadores de rendimiento
    print("\n Indicadores de rendimiento:")
    report = classification_report(
        all_labels, 
        all_preds, 
        target_names=class_names,
        digits=4,
    )
    print(report)                                                                           # Imprime los indicadores de rendimiento
    
# Evaluar el modelo
evaluate_model(model, test_loader)                                                # Evalúa el modelo


### c. Matriz de confusión e indicadores de desempeño

#### Sin data augmentation:
La matriz de confusion al evaluar el mejor modelo obtenido con data augmentation es la siguiente:

<div style="text-align: center;">
    <img src="Imagenes_lab3\matriz.png" width="40%">
</div>

<p style="text-align: justify; line-height: 1.5;">
La matriz muestra valores máximos de 50 en la diagonal principal (predicciones correctas por clase), indicando que el modelo identificó correctamente un número significativo de imágenes por clase. Las confusiones son mínimas y están principalmente distribuidas entre clases con caractéristicas visuales similares. <br>
Estos valores demuestran que el modelo es robusto, incluso en clases más desafiantes.
 

Los ingicadores evaluados (precision, recall y F1-score) son por clase y muestran los siguientes resultados para los 21 clases:  <br>
1. Precisión: Mide la proporción de predicciones correctas por clase respecto al total de predicciones hechas para esa clase. En este caso se obtuvo una media y macro ponderada de 0.9757  <br>
2. Sensibilidad (Recall): Mide la proporción de predicciones correctas para una clase respecto al total de muestras reales de esa clase. La media macro y ponderada fue de un 0.9752<br>
3. F1-score: Es la media armónica entre precisión y sensibilida, proporcionando un balance entre ambas métricas. La media y macro ponderada fue de 0.9752<br>
4. Exactitud global (Accuracy): El modelo logró una exactitud de 97.52% en el conjunto de prueba.<br>
</p>


Indicadores de rendimiento:
<div style="display: flex; justify-content: center;">
<table>
    <thead>
        <tr>
            <th>Clase</th>
            <th>Precision</th>
            <th>Recall</th>
            <th>F1-Score</th>
            <th>Support</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>agricultural</td>
            <td>1.0000</td>
            <td>1.0000</td>
            <td>1.0000</td>
            <td>50</td>
        </tr>
        <tr>
            <td>airplane</td>
            <td>1.0000</td>
            <td>1.0000</td>
            <td>1.0000</td>
            <td>50</td>
        </tr>
        <tr>
            <td>baseballdiamond</td>
            <td>0.9792</td>
            <td>0.9400</td>
            <td>0.9592</td>
            <td>50</td>
        </tr>
        <tr>
            <td>beach</td>
            <td>0.9800</td>
            <td>0.9800</td>
            <td>0.9800</td>
            <td>50</td>
        </tr>
        <tr>
            <td>buildings</td>
            <td>0.9592</td>
            <td>0.9400</td>
            <td>0.9495</td>
            <td>50</td>
        </tr>
        <tr>
            <td>chaparral</td>
            <td>1.0000</td>
            <td>1.0000</td>
            <td>1.0000</td>
            <td>50</td>
        </tr>
        <tr>
            <td>denseresidential</td>
            <td>0.9796</td>
            <td>0.9600</td>
            <td>0.9697</td>
            <td>50</td>
        </tr>
        <tr>
            <td>forest</td>
            <td>0.9804</td>
            <td>1.0000</td>
            <td>0.9901</td>
            <td>50</td>
        </tr>
        <tr>
            <td>freeway</td>
            <td>0.9615</td>
            <td>1.0000</td>
            <td>0.9804</td>
            <td>50</td>
        </tr>
        <tr>
            <td>golfcourse</td>
            <td>0.9231</td>
            <td>0.9600</td>
            <td>0.9412</td>
            <td>50</td>
        </tr>
        <tr>
            <td>harbor</td>
            <td>0.9804</td>
            <td>1.0000</td>
            <td>0.9901</td>
            <td>50</td>
        </tr>
        <tr>
            <td>intersection</td>
            <td>0.9245</td>
            <td>0.9800</td>
            <td>0.9515</td>
            <td>50</td>
        </tr>
        <tr>
            <td>mediumresidential</td>
            <td>0.9216</td>
            <td>0.9400</td>
            <td>0.9307</td>
            <td>50</td>
        </tr>
        <tr>
            <td>mobilehomepark</td>
            <td>0.9615</td>
            <td>1.0000</td>
            <td>0.9804</td>
            <td>50</td>
        </tr>
        <tr>
            <td>overpass</td>
            <td>1.0000</td>
            <td>0.9800</td>
            <td>0.9899</td>
            <td>50</td>
        </tr>
        <tr>
            <td>parkinglot</td>
            <td>1.0000</td>
            <td>1.0000</td>
            <td>1.0000</td>
            <td>50</td>
        </tr>
        <tr>
            <td>river</td>
            <td>0.9787</td>
            <td>0.9200</td>
            <td>0.9485</td>
            <td>50</td>
        </tr>
        <tr>
            <td>runway</td>
            <td>1.0000</td>
            <td>1.0000</td>
            <td>1.0000</td>
            <td>50</td>
        </tr>
        <tr>
            <td>sparseresidential</td>
            <td>0.9796</td>
            <td>0.9600</td>
            <td>0.9697</td>
            <td>50</td>
        </tr>
        <tr>
            <td>storagetanks</td>
            <td>1.0000</td>
            <td>0.9400</td>
            <td>0.9691</td>
            <td>50</td>
        </tr>
        <tr>
            <td>tenniscourt</td>
            <td>0.9800</td>
            <td>0.9800</td>
            <td>0.9800</td>
            <td>50</td>
        </tr>
        <tr>
            <td><strong>Overall</strong></td>
            <td>0.9757</td>
            <td>0.9752</td>
            <td>0.9752</td>
            <td>1050</td>
        </tr>
    </tbody>
</table>
</div>

A partir de la tabla se pueden tener algunas observaciones respecto a los resultados:
1. El desempeño es consistente entre clases debido al buen ajuste del modelo y el uso de GoogLeNet preentrenado, optimizado con Adam y un criterio de detención adecuado.
2. Las clases como parkinglot, runway y chaparral lograron un F1-score perfecto, mientras que clases como golfcourse y mediumresidential tienen valores ligeramente más bajos. Esto podría ser porque estas clases comparten carácteristicas visuales con otras categorias, lo que complica su clasificación.
3. A pesar de no utilizar data augmentation, el modelo logró un desempeño sobresaliente. Esto sugiere que el conjunto de datos de entrenamiento tiene buena diversidad y calidad, proporcionando suficiente informacion para generalizar.

#### Con data augmentation:

Se obtuvo la siguiente matriz de confusión al probar el codigo con data augmentation
<div style="text-align: center;">
    <img src="Imagenes_lab3\matriz con da.png" width="40%">
</div>

Se puede ver que en este caso aunque se introduce mas variabilidad en los datos en el momento del entrenamiento, también genera ligeros errores en clases que previamente tenían un desempeño perfecto, como lo son las clases de tenniscourt y storagetanks que muestran ligeras reducciones en recall y presición con data augmentation.


<div style="display: flex; justify-content: center;">
<table>
  <thead>
    <tr>
      <th>Clase</th>
      <th>Precisión</th>
      <th>Recall</th>
      <th>F1-Score</th>
      <th>Support</th>
    </tr>
  </thead>
  <tbody>
    <tr><td>agricultural</td><td>1.0000</td><td>1.0000</td><td>1.0000</td><td>50</td></tr>
    <tr><td>airplane</td><td>1.0000</td><td>0.9800</td><td>0.9899</td><td>50</td></tr>
    <tr><td>baseballdiamond</td><td>1.0000</td><td>0.9600</td><td>0.9796</td><td>50</td></tr>
    <tr><td>beach</td><td>0.9796</td><td>0.9600</td><td>0.9697</td><td>50</td></tr>
    <tr><td>buildings</td><td>0.9388</td><td>0.9200</td><td>0.9293</td><td>50</td></tr>
    <tr><td>chaparral</td><td>1.0000</td><td>1.0000</td><td>1.0000</td><td>50</td></tr>
    <tr><td>denseresidential</td><td>0.8727</td><td>0.9600</td><td>0.9143</td><td>50</td></tr>
    <tr><td>forest</td><td>0.9245</td><td>0.9800</td><td>0.9515</td><td>50</td></tr>
    <tr><td>freeway</td><td>0.9259</td><td>1.0000</td><td>0.9615</td><td>50</td></tr>
    <tr><td>golfcourse</td><td>0.9796</td><td>0.9600</td><td>0.9697</td><td>50</td></tr>
    <tr><td>harbor</td><td>1.0000</td><td>1.0000</td><td>1.0000</td><td>50</td></tr>
    <tr><td>intersection</td><td>0.8929</td><td>1.0000</td><td>0.9434</td><td>50</td></tr>
    <tr><td>mediumresidential</td><td>1.0000</td><td>0.8200</td><td>0.9011</td><td>50</td></tr>
    <tr><td>mobilehomepark</td><td>1.0000</td><td>0.9600</td><td>0.9796</td><td>50</td></tr>
    <tr><td>overpass</td><td>0.9583</td><td>0.9200</td><td>0.9388</td><td>50</td></tr>
    <tr><td>parkinglot</td><td>1.0000</td><td>1.0000</td><td>1.0000</td><td>50</td></tr>
    <tr><td>river</td><td>0.9608</td><td>0.9800</td><td>0.9703</td><td>50</td></tr>
    <tr><td>runway</td><td>0.9804</td><td>1.0000</td><td>0.9901</td><td>50</td></tr>
    <tr><td>sparseresidential</td><td>0.9800</td><td>0.9800</td><td>0.9800</td><td>50</td></tr>
    <tr><td>storagetanks</td><td>0.9216</td><td>0.9400</td><td>0.9307</td><td>50</td></tr>
    <tr><td>tenniscourt</td><td>1.0000</td><td>0.9600</td><td>0.9796</td><td>50</td></tr>
  </tbody>
  <tfoot>
    <tr>
      <td><strong>Overall</strong></td>
      <td>0.9674</td>
      <td>0.9657</td>
      <td>0.9657</td>
      <td>1050</td>
    </tr>
  </tfoot>
</table>
</div>

Al observar los datos de la tabla se pueden ver pequeñas diferencias con respecto al modelo sin data augmentation, que se profundizan en los siguientes puntos:
1. Precision (97.52% sin da y 96.57% con da): Aunque la precisión global con data augmentation es ligeramente menor, este resultado no necesariamente indica un empeoramiento, ya que el data augmentation tiende a mejorar la capacidad de generalizacion del modelo en datos no vistos.
2. Recall y F1-score: En general el data augmentation no impactó de forma significativa en las clases que ya tenian un desempeño perfecto o casi perfecto, pero podría haber mejorado la generalización para las clases más dificiles en un conjunto diferente de validación.
3. El uso de data augmentation introduce variaciones en las imágenes de entrenamiento, como rotaciones, reflejos y cambios de color, que buscan simular imagenes que no se encuentren en el conjunto de entrenamiento pero si en el mundo real. Esto en general es útil para mejorar la robustez del modelo pero en este caso puede haber causado una ligera disminucion del desempeño. Esto se puede atibuir tanto a un posible sobreentrenamiento del modelo o a que los datos de prueba no eran los suficientemente variado para reflejar de forma adecuada los escenarios para los que fue entrenado el modelo con las técnicas de data augmentation. 




## 4. Prueba de los modelos con imagenese de Google Earth Engine (GEE).

Se obtienen 10 imagenes utilizando google earth, para ello se seleccionarion diferentes puntos de la región metropolitana y sus alrededores, teniendo en concreto las siguientes imagenes:

<div style="display: flex; flex-wrap: wrap; justify-content: center;">

<img src="Imagenes_lab3\google\1.png" alt="Imagen 1" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\google\2.png" alt="Imagen 2" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\google\3.png" alt="Imagen 3" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\google\4.png" alt="Imagen 4" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\google\5.png" alt="Imagen 5" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\google\6.png" alt="Imagen 6" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\google\7.png" alt="Imagen 7" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\google\8.png" alt="Imagen 8" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\google\9.png" alt="Imagen 9" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\google\10.png" alt="Imagen 9" width="300" style="margin: 5px;">

</div>

Luego se realiza la clasificiación de cada imagen utilizando tanto el modelo sin data augmentation como con data augmentation.

In [ ]:
import os
import torch
import torchvision.models as models
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
from torch import nn
from collections import Counter

# Ruta de la carpeta de imágenes
Image_folder = "C:\\Users\\tiari\\OneDrive\\Documents\\IA\\Laboratorios-de-inteligencia-artificial\\Imagenes_lab3\\google"

# Transformaciones de los datos
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Función para cargar imágenes de una carpeta
def load_images_from_folder(folder):
    images = []
    file_names = []
    for file in os.listdir(folder):
        if file.endswith(".png"):
            img = Image.open(os.path.join(folder, file))
            img = img.convert("RGB")  # Convert image to RGB
            img = transform(img)
            images.append(img)
            file_names.append(file)
    return images, file_names

# Cargar las imágenes de la carpeta
images, file_names = load_images_from_folder(Image_folder)

# Selección de la GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Dispositivo:", "cuda" if torch.cuda.is_available() else "cpu")

# Cargar el modelo preentrenado GoogLeNet
model = models.googlenet(pretrained=True)
num_classes = 21
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("C:\\Users\\tiari\\OneDrive\\Documents\\IA\\modelos\\model_sin_da_epoch_8.pth"))
model = model.to(device)
model.eval()

# Inicializa la lista de predicciones
predictions = []
with torch.no_grad():
    for img in images:
        img = img.unsqueeze(0).to(device)
        output = model(img)
        _, predicted = torch.max(output, 1)
        predictions.append(predicted.item())

# Nombres de las clases
class_names = [
    'agricultural', 'airplane', 'baseballdiamond', 'beach', 'buildings',
    'chaparral', 'denseresidential', 'forest', 'freeway', 'golfcourse',
    'harbor', 'intersection', 'mediumresidential', 'mobilehomepark', 'overpass',
    'parkinglot', 'river', 'runway', 'sparseresidential', 'storagetanks', 'tenniscourt'
]

# Cuenta las predicciones por clase
class_counts = {class_name: predictions.count(i) for i, class_name in enumerate(class_names)}
total_images = len(predictions)

# Calcula el porcentaje de predicciones por clase
class_percentages = Counter(predictions)
top_5_classes = sorted(class_percentages.items(), key=lambda x: x[1], reverse=True)[:5]

# Porcentajes de las 5 clases con mayor representación
top_5_percentages = [(class_names[class_id], count / total_images * 100) for class_id, count in top_5_classes]
print("Porcentajes de las 5 clases con mayor representación:")
for class_name, percentage in top_5_percentages:
    print(f"{class_name}: {percentage:.2f}%")

# Diccionario con las etiquetas verdaderas
true_labels = {
    "1.png": "agricultural",
    "2.png": "denseresidential",
    "3.png": "sparseresidential",
    "4.png": "chaparral",
    "5.png": "airplane",
    "6.png": "overpass",
    "7.png": "river",
    "8.png": "tenniscourt",
    "9.png": "forest",
    "10.png": "agricultural",
}

# Calcular el porcentaje de acierto
correct_predictions = 0
total_images = len(images)

results = []
for i, file_name in enumerate(file_names):
    predicted_class = class_names[predictions[i]]
    true_class = true_labels[file_name]
    is_correct = predicted_class == true_class
    correct_predictions += is_correct
    results.append((file_name, true_class, predicted_class, is_correct))

accuracy = correct_predictions / total_images * 100
print(f"Porcentaje de acierto del modelo: {accuracy:.2f}%")

print("Predicciones por clase:")

for file_name, true_class, predicted_class, is_correct in results:
    print(f"Imagen: {file_name}, Clase verdadera: {true_class}, Predicción: {predicted_class}, Correcta: {is_correct}")
    
# Mostrar las imágenes con sus predicciones
for i, (image, file_name) in enumerate(zip(images, file_names)):
    image = image.permute(1, 2, 0).numpy()
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"Predicción: {class_names[predictions[i]]}\nVerdadera: {true_labels[file_name]}", fontsize=12)
    plt.show()





#### Con data augmentation

<p style="text-align: justify; line-height: 1.5;">
Se comienza realizando la clasificación utilizando el modelo sin data augmentation, obteniendo:<br>

<b> Porcentajes de las 5 clases con mayor representación:</b>
- overpass: 30.00%
- tenniscourt: 20.00%
- chaparral: 20.00%
- baseballdiamond: 10.00%
- denseresidential: 10.00%<br>

<b> Porcentaje de acierto del modelo: </b> 40.00%

<b>Predicciones por clase:</b>


<div style="display: flex; justify-content: center;">
<table>
  <thead>
    <tr>
      <th>Imagen</th>
      <th>Clase Verdadera</th>
      <th>Predicción</th>
      <th>Correcta</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>agricultural</td>
      <td>overpass</td>
      <td>False</td>
    </tr>
    <tr>
      <td>10</td>
      <td>agricultural</td>
      <td>baseballdiamond</td>
      <td>False</td>
    </tr>
    <tr>
      <td>2</td>
      <td>denseresidential</td>
      <td>denseresidential</td>
      <td>True</td>
    </tr>
    <tr>
      <td>3</td>
      <td>sparseresidential</td>
      <td>tenniscourt</td>
      <td>False</td>
    </tr>
    <tr>
      <td>4</td>
      <td>chaparral</td>
      <td>chaparral</td>
      <td>True</td>
    </tr>
    <tr>
      <td>5</td>
      <td>airplane</td>
      <td>buildings</td>
      <td>False</td>
    </tr>
    <tr>
      <td>6</td>
      <td>overpass</td>
      <td>overpass</td>
      <td>True</td>
    </tr>
    <tr>
      <td>7</td>
      <td>river</td>
      <td>overpass</td>
      <td>False</td>
    </tr>
    <tr>
      <td>8</td>
      <td>tenniscourt</td>
      <td>tenniscourt</td>
      <td>True</td>
    </tr>
    <tr>
      <td>9</td>
      <td>forest</td>
      <td>chaparral</td>
      <td>False</td>
    </tr>
  </tbody>
</table>
</div>

<b> Inspección visual: </b>  

<div style="display: flex; flex-wrap: wrap; justify-content: center;">
<img src="Imagenes_lab3\Class_sin_da\1.png" alt="Imagen 1" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\Class_sin_da\2.png" alt="Imagen 2" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\Class_sin_da\3.png" alt="Imagen 3" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\Class_sin_da\4.png" alt="Imagen 4" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\Class_sin_da\5.png" alt="Imagen 5" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\Class_sin_da\6.png" alt="Imagen 6" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\Class_sin_da\7.png" alt="Imagen 7" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\Class_sin_da\8.png" alt="Imagen 8" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\Class_sin_da\9.png" alt="Imagen 9" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\Class_sin_da\10.png" alt="Imagen 9" width="300" style="margin: 5px;">
</div>

#### Sin data augmentation

<p style="text-align: justify; line-height: 1.5;">
<b> Porcentajes de las 5 clases con mayor representación (predicción):</b>

- runway: 20.00%
- denseresidential: 20.00%
- chaparral: 20.00%
- sparseresidential: 10.00%
- buildings: 10.00%

<b> Porcentaje de acierto del modelo: </b> 50.00%<br>
<b> Predicciones por clase:</b>
</p>
<div style="display: flex; justify-content: center;">
<table>
  <thead>
    <tr>
      <th>Imagen</th>
      <th>Clase Verdadera</th>
      <th>Predicción</th>
      <th>Correcta</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>agricultural</td>
      <td>runway</td>
      <td>False</td>
    </tr>
    <tr>
      <td>10</td>
      <td>agricultural</td>
      <td>runway</td>
      <td>False</td>
    </tr>
    <tr>
      <td>2</td>
      <td>denseresidential</td>
      <td>denseresidential</td>
      <td>True</td>
    </tr>
    <tr>
      <td>3</td>
      <td>sparseresidential</td>
      <td>sparseresidential</td>
      <td>True</td>
    </tr>
    <tr>
      <td>4</td>
      <td>chaparral</td>
      <td>chaparral</td>
      <td>True</td>
    </tr>
    <tr>
      <td>5</td>
      <td>airplane</td>
      <td>buildings</td>
      <td>False</td>
    </tr>
    <tr>
      <td>6</td>
      <td>overpass</td>
      <td>overpass</td>
      <td>True</td>
    </tr>
    <tr>
      <td>7</td>
      <td>river</td>
      <td>denseresidential</td>
      <td>False</td>
    </tr>
    <tr>
      <td>8</td>
      <td>tenniscourt</td>
      <td>tenniscourt</td>
      <td>True</td>
    </tr>
    <tr>
      <td>9</td>
      <td>forest</td>
      <td>chaparral</td>
      <td>False</td>
    </tr>
  </tbody>
</table>

</div>

<b> Inspección visual:</b>

<div style="display: flex; flex-wrap: wrap; justify-content: center;">
<img src="Imagenes_lab3\Class_con_da\1.png" alt="Imagen 1" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\Class_con_da\2.png" alt="Imagen 2" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\Class_con_da\3.png" alt="Imagen 3" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\Class_con_da\4.png" alt="Imagen 4" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\Class_con_da\5.png" alt="Imagen 5" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\Class_con_da\6.png" alt="Imagen 6" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\Class_con_da\7.png" alt="Imagen 7" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\Class_con_da\8.png" alt="Imagen 8" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\Class_con_da\9.png" alt="Imagen 9" width="300" style="margin: 5px;">
<img src="Imagenes_lab3\Class_con_da\10.png" alt="Imagen 9" width="300" style="margin: 5px;">
</div>


##### Comparativa de los resultados entregados por ambos modelos.

- Se puede ver como a pesar de que en las pruebas realizadas anteriormente el modelo con data augmentation parecia ser peor, en este caso aumento el porcentaje de acierto en un 10% (40% a 50%). Esto sugiere que la diversidad introducida en los datos mejora la capacidad del modelo para generalizar y enfrentarce a clasificar imagenes mas desafiantes, sobretodo teniendo en cuenta que era muy dificil que cualquier modelo fuese capaz de detectar al rio mapocho como rio por ejemplo.<br>
- En cuanto a la distribución de los errores, algunos persistieron con o sin data augmentation, lo que podría indicar limitaciones en las carácteristicas aprendidas por el modelo o en la representacion de estas clases en el conjunto de datos original. Por ejemplo en la imagen del aeropuerto ningún modelo es capaz de detectar los aviones, lo que se asume que es debido a que igual se presentan edificios en la imagen y los modelos estaban entrenados para detectar los aviones a una mayor cercania.<br>
- Se pudo ver tambien como las clases de chaparral y tenniscourt tuvieron un buen desempeño con ambos modelos, lo que sugiere que estas clases tienen patrones mas distintivos. A diferencia de las clases como agricultural y river que confunden al modelo con otras visualmente similares (runway y denseresidential).